In [1]:
import ee
import geemap
from geeCenterline import *
ee.Initialize()

# Centerline extraction of the Tallahatchie River
This notebook provides an example of how to extract the centerline of river from remote sensing images. In this example, we are going to use the surface reflectance images provided by Landsat and PlanetScope. To concentrate on the workflow of centerline extraction, we have already upload images we need to assets of Google Earth Engine. However, users of Google Earth Engine can obtain Landsat collection directly. Instructions to upload Landsat images through Google Earth Engine is in [here](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2).

# Import images
AssetIDs of images going to be uploaded are required and images should includes at least four bands: blue, green, red, and near-infrared, and the short-wave near-infrared band is also recommended to be included if avaliable. Names of these bands should be uniformed as 'Red', 'Green', 'Blue', 'NIR', 'SWIR1'.

In [2]:
# assits ids of images and region of interest
PS_id = 'users/luoyee1997/example/Tallahatchie_Planet'
LS_id = 'users/luoyee1997/example/Tallahatchie_Landsat2'
roi_id = 'projects/ee-alpha-luoyee1997/assets/LT'

In [3]:
# band names of PlanetScope
bands_in_PS = ['b1', 'b2', 'b3', 'b4']
# required names
bandnames_PS = ['Blue', 'Green', 'Red', 'NIR']
# band names of Landsat
bands_in_LS = ['SR_B4', 'SR_B3', 'SR_B2', 'SR_B5', 'SR_B6']
# required names
bandnames_LS = ['Red', 'Green', 'Blue', 'NIR', 'SWIR1']
roi = load_file(roi_id)
imgLS = load_file(LS_id, bands_in_LS, bandnames_LS).clip(roi)
imgPS = load_file(PS_id, bands_in_PS, bandnames_PS).clip(roi)

# Obtain river mask from Landsat image
Comparing with PlanetScope, Landsat has lower resolution, which means lower computation. We use the Landsat image and the river mask obtained from it to filter out most unwanted water surfaces from the region of interest. In this section, we are going to
1. Identify water surfaces.
2. Identify the river from water surfaces.
3. Create a mask for the PlanetScope image to locate the river.

## Classification
**thrs** of the classification function are the thresholds to classify water, bare soil and vegetation from the NDVI grey image, where values less than thr2 are considered as water. The value between thr3 and thr4 are considered as bare soil, and the value larger than thr5 are considered as vegetation. One thing should be noticed is that we are not going to use these threshold to classify the image. Instead, this classification are going to be used to select training set for the classifer. Thus, these thresholds are not accurite values. The threshold can be modified if the classifer does not perform well, but the default values are robust basing on our tests.

**numPts** is the parameter to decide the number of points which are going to be selected as the training set.

In [4]:
# landsat classification
classLS = classification(imgLS, thr2=0, thr3=0.2, thr4=0.3, thr5=0.6, numPts=100)

## River mask
The river is identified and the mask is dilated for PlanetScope.

In [5]:
waterLS = classLS.eq(1).clip(roi)

# connect the water masks divided by small gaps. The radius is the half scale of the gap to be filled.
waterLS1 = close(waterLS, radius=1.5, kernelType='square')

# identify the river from other water masks. The second input is the minimum area that is considered as the river.
# The second input is the connectivity of pixels. 4 means 4-connectivity and 8 means 8-connectivity.
riverLS = noise_removal(waterLS1, 500, 8)

# Dilate the river mask
riverMask = mask_hrs(riverLS, r=1.5, i=3)

# Obtain river mask from the PlanetScope image
This section demonstrates how to mask the clasified PlanetScope image and obtain a simply connected river mask.

In [6]:
# planet classification
classPS = classification(imgPS, thr2=0, thr3=0.2, thr4=0.3, thr5=0.6, numPts=100)

In [7]:
# Obtain the river mask
waterPS = classPS.eq(1).updateMask(riverMask)
waterPS2 = close(waterPS, radius=3)
riverPS = noise_removal(waterPS2, 1000, 4)

# fill holes of river masks
notRiver = riverPS.unmask().Not()
riverPS1 = noise_removal(notRiver, 50, 4)
riverPS2 = riverPS1.unmask().Not()

# Centerline extraction

## Medial Axis Transfrom (MAT)
1. The distance from the river pixels to the nearest non-river pixel is measured.
2. Compute the gradient of distance.
3. Thresholding out the centerline. Pixels with a value less than the given value is considered as centerline.

In [8]:
# calculate centerline, thre is the threshold to decide wether a value will be considered as centerline.
cl = CalculateCenterline1(riverPS2.unmask(), thre=0.9)

## Remove noises caused by MAT and trim unwanted branches

In [9]:
# trim unwanted branches, the second input is the longest distance that a line will be considered as branches.
# The third input is the number of iteration to do the trimming algrithm.
trimed = trim_centerline(cl, 300, 5).selfMask()

# Out put
The result is going to be output as an asset in Google Earth Engine. This process may take a long time. Please check the process of the test at the Code Editor of Google Earthe Engine.

In [10]:
# the location to output the centerline. PLEASE use our own space!
out_loc = 'users/luoyee1997/example/trimed'
task = ee.batch.Export.image.toAsset(**{
    'image': trimed,
    'description': 'river_centerline',
    'assetId': out_loc,
    'scale': 3,
    'maxPixels': 6e8,
  'region': roi
})
task.start()

# Visualize the image
Visualizing image can help you to check whether the parameters you selected is suitable.

In [11]:
Map = geemap.Map()
vis = {
    'bands': ['Red', 'Green', 'Blue'],
    'min': 0,
    'max': 0.7042,
    'gamma': 2
}
Map.centerObject(imgLS, 12)
Map.addLayer(imgLS, vis, 'LS')

In [12]:
# classified Landsat iamge
Map.addLayer(classLS, {'min':1, 'max':3, 'palette': ['#0080FF', 'red', 'green']}, 'classLS')
# the river mask obtained from Landsat
Map.addLayer(riverMask, {}, 'mask')
# classified PlanetScope image
Map.addLayer(classPS, {'min':1, 'max':3, 'palette': ['#0080FF', 'red', 'green']}, 'classPS')
# river mask to compute the centerline.
Map.addLayer(riverPS, {}, 'riverPS')

In [13]:
# upload the computed centerline and visualize it
trimed = ee.Image(out_loc)
Map.addLayer(trimed.selfMask(), {}, 'cl')

In [14]:
Map

Map(center=[33.519505219659685, -90.29477822219337], controls=(WidgetControl(options=['position', 'transparent…